In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
%matplotlib inline

In [2]:
path = 'data/merged_congestion_dummy.csv'
data = pd.read_csv(path)
data.dropna()
data.head()

tti  speed  precipitation  tempurature  hour_0  hour_1  hour_2  hour_3  \
0  1.362      7            0.1           29       1       0       0       0   
1  1.283      7            0.1           29       1       0       0       0   
2  1.125      7            0.1           29       1       0       0       0   
3  1.352      7            0.1           29       1       0       0       0   
4  1.330      7            0.1           29       1       0       0       0   

   hour_4  hour_5  ...  small_section_idx_88417295b1fffff  \
0       0       0  ...                                  0   
1       0       0  ...                                  0   
2       0       0  ...                                  0   
3       0       0  ...                                  0   
4       0       0  ...                                  0   

   small_section_idx_88417295c1fffff  small_section_idx_88417295c3fffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295c7fffff  small_section_idx_88417295cbfffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295cdfffff  small_section_idx_88417295d1fffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295e1fffff  small_section_idx_88417295e3fffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295e7fffff  
0                                  0  
1                                  0  
2                                  0  
3                                  0  
4                                  0  

[5 rows x 576 columns]

In [3]:
d1 = data.drop(columns=["tti", 'precipitation']).astype('int8')
d2 = data[['tti','precipitation']].astype('float16')
d1.shape, d2.shape, d1.dtypes, d2.dtypes

((372973, 574), (372973, 2), speed                                int8
 tempurature                          int8
 hour_0                               int8
 hour_1                               int8
 hour_2                               int8
                                      ... 
 small_section_idx_88417295cdfffff    int8
 small_section_idx_88417295d1fffff    int8
 small_section_idx_88417295e1fffff    int8
 small_section_idx_88417295e3fffff    int8
 small_section_idx_88417295e7fffff    int8
 Length: 574, dtype: object, tti              float16
 precipitation    float16
 dtype: object)

In [4]:
data = pd.concat([d2,d1], axis=1, sort=False)
data.shape, data.dtypes

((372973, 576), tti                                  float16
 precipitation                        float16
 speed                                   int8
 tempurature                             int8
 hour_0                                  int8
                                       ...   
 small_section_idx_88417295cdfffff       int8
 small_section_idx_88417295d1fffff       int8
 small_section_idx_88417295e1fffff       int8
 small_section_idx_88417295e3fffff       int8
 small_section_idx_88417295e7fffff       int8
 Length: 576, dtype: object)

In [5]:
data.head()

tti  precipitation  speed  tempurature  hour_0  hour_1  hour_2  \
0  1.362305       0.099976      7           29       1       0       0   
1  1.283203       0.099976      7           29       1       0       0   
2  1.125000       0.099976      7           29       1       0       0   
3  1.351562       0.099976      7           29       1       0       0   
4  1.330078       0.099976      7           29       1       0       0   

   hour_3  hour_4  hour_5  ...  small_section_idx_88417295b1fffff  \
0       0       0       0  ...                                  0   
1       0       0       0  ...                                  0   
2       0       0       0  ...                                  0   
3       0       0       0  ...                                  0   
4       0       0       0  ...                                  0   

   small_section_idx_88417295c1fffff  small_section_idx_88417295c3fffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295c7fffff  small_section_idx_88417295cbfffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295cdfffff  small_section_idx_88417295d1fffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295e1fffff  small_section_idx_88417295e3fffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295e7fffff  
0                                  0  
1                                  0  
2                                  0  
3                                  0  
4                                  0  

[5 rows x 576 columns]

In [6]:
X = data.drop(columns=["tti"]).values
Y = data[['tti']].values

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=10)

In [8]:
std = StandardScaler()
std_x = std.fit(X_train[:,[0,1,2]])
X_train[:,[0,1,2]] = std_x.transform(X_train[:,[0,1,2]])
X_test[:,[0,1,2]] = std_x.transform(X_test[:,[0,1,2]])
std_y = std.fit(y_train)
y_train = std_y.transform(y_train)
y_test = std_y.transform(y_test)

In [9]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('dt',  DecisionTreeRegressor()),
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('dt',
                 DecisionTreeRegressor(criterion='mse', max_depth=None,
                                       max_features=None, max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       presort=False, random_state=None,
                                       splitter='best'))],
         verbose=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {                            # specify the degree of the polynomial here
    'dt__max_depth'   : np.arange(2, 40)   # search range for parameter α
                                           # in the sgd step of the pipeline
}

# set-up the grid-search
grid_search = GridSearchCV(
    pipeline,                             # the ML pipeline we wish to test
    parameters,                              # all parameter settings
    scoring='neg_mean_squared_error',
    cv=5,
                                      # since this maximizes, we use -MSE
    return_train_score=True           # useful for debugging, calculate the trainset
                                      # performance
)

# start the gridsearch
grid_search.fit(X_train,y_train);
# y_predict = pipeline.predict(X_test)
print ("best score:", grid_search.best_score_, "best params:", grid_search.best_params_)

In [10]:
X_t, X_v, y_t, y_v = train_test_split(X_train,y_train,test_size=0.2,random_state=10)
from sklearn.metrics import mean_squared_error
pipeline.set_params(dt__max_depth=2)
pipeline.fit(X_t, y_t)
predict_t = pipeline.predict(X_t)
MSE_t = mean_squared_error(predict_t, y_t)
predict = pipeline.predict(X_v)
MSE = mean_squared_error(predict, y_v)
MSE_t, MSE

(0.9271076441098651, 1.2318549992818453)

In [ ]:
from sklearn.metrics import mean_squared_error
predict = pipeline.predict(X_test)
MSE = mean_squared_error(predict, y_test)
MSE

In [ ]:
with open("DT_congestion_result.txt", 'w') as result:
    result.write("best score: " + str(grid_search.best_score_) + "\nbest params:" + str(grid_search.best_params_) + '\ntest MSE: ' + str(MSE))